In [ ]:
from sklearn.metrics import mean_squared_error
import seaborn as sns

import matplotlib.pyplot as plt

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(user_item_matrix_normalized.values.flatten(), 
                                predicted_matrix.flatten()))

# Create a comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot original matrix
sns.heatmap(user_item_matrix_normalized.iloc[:50, :], ax=ax1, cmap='YlOrRd')
ax1.set_title('Original Matrix (First 50 Users)')

# Plot predicted matrix
sns.heatmap(predicted_matrix[:50, :], ax=ax2, cmap='YlOrRd')
ax2.set_title('Predicted Matrix (First 50 Users)')

plt.suptitle(f'Original vs Predicted Matrix Comparison\nRMSE: {rmse:.4f}')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, mean_absolute_error
import numpy as np

# Convert predictions to binary (1 if value > threshold, 0 otherwise)
threshold = 0.5
y_true = (user_item_matrix_normalized.values > threshold).astype(int)
y_pred = (predicted_matrix > threshold).astype(int)

# Calculate classical metrics
precision = precision_score(y_true.flatten(), y_pred.flatten())
recall = recall_score(y_true.flatten(), y_pred.flatten()) 
f1 = f1_score(y_true.flatten(), y_pred.flatten())

# Calculate error metrics
mae = mean_absolute_error(user_item_matrix_normalized.values.flatten(), predicted_matrix.flatten())
rmse = np.sqrt(mean_squared_error(user_item_matrix_normalized.values.flatten(), predicted_matrix.flatten()))

# Function to calculate MAP
def mean_average_precision(y_true, y_pred, k=10):
    ap_scores = []
    for i in range(len(y_true)):
        true_items = np.where(y_true[i] > threshold)[0]
        pred_items = np.argsort(y_pred[i])[::-1][:k]
        if len(true_items) > 0:
            ap = 0
            hits = 0
            for j, item in enumerate(pred_items):
                if item in true_items:
                    hits += 1
                    ap += hits / (j + 1)
            ap_scores.append(ap / min(len(true_items), k))
    return np.mean(ap_scores)

# Function to calculate NDCG
def ndcg(y_true, y_pred, k=10):
    ndcg_scores = []
    for i in range(len(y_true)):
        idcg = dcg(np.sort(y_true[i])[::-1][:k])
        if idcg > 0:
            ndcg_scores.append(dcg(y_pred[i][np.argsort(y_true[i])[::-1][:k]]) / idcg)
    return np.mean(ndcg_scores)

def dcg(scores):
    return np.sum(scores / np.log2(np.arange(2, len(scores) + 2)))

# Calculate ranking metrics
map_score = mean_average_precision(user_item_matrix_normalized.values, predicted_matrix)
ndcg_score = ndcg(user_item_matrix_normalized.values, predicted_matrix)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAP@10: {map_score:.4f}")
print(f"NDCG@10: {ndcg_score:.4f}")